In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
print('Pytorch version :', torch.__version__)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Device :',device)

Pytorch version : 1.7.1
Device : cuda:0


In [2]:
input_size = 224
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
])

In [3]:
train_data = datasets.CIFAR100(root='data',train=True,transform=transform,download=True)
test_data = datasets.CIFAR100(root='data',train=False,transform=transform,download=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
train_data.__getitem__(0)[0].size()

torch.Size([3, 224, 224])

In [6]:
BATCH_SIZE = 64
train_iter = DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=1)
test_iter = DataLoader(test_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=1)

In [7]:
import torch
import torch.nn as nn

class VGG(nn.Module):
    def __init__(self, num_classes=100):
        super(VGG, self).__init__()

        self.relu = nn.ReLU(inplace=True)

        # Convolution Feature Extraction Part
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2   = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3_1   = nn.BatchNorm2d(256)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.bn3_2   = nn.BatchNorm2d(256)
        self.pool3   = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4_1   = nn.BatchNorm2d(512)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn4_2   = nn.BatchNorm2d(512)
        self.pool4   = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn5_1   = nn.BatchNorm2d(512)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn5_2   = nn.BatchNorm2d(512)
        self.pool5   = nn.MaxPool2d(kernel_size=2, stride=2)
        '''======================== TO DO (1) ========================'''
        '''==========================================================='''


        # Fully Connected Classifier Part
        self.gap      = nn.AdaptiveAvgPool2d(1)
        self.fc1      = nn.Linear(512 * 1 * 1, 4096)
        self.dropout1 = nn.Dropout()

        '''==========================================================='''
        '''======================== TO DO (2) ========================'''
        self.fc2      = nn.Linear(4096,4096)
        self.dropout2 = nn.Dropout()

        self.fc3      = nn.Linear(4096,100)
        '''======================== TO DO (2) ========================'''
        '''==========================================================='''

    def forward(self, x):
        # Convolution Feature Extraction Part
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool2(x)

        x = self.conv3_1(x)
        x = self.bn3_1(x)
        x = self.relu(x)
        x = self.conv3_2(x)
        x = self.bn3_2(x)
        x = self.relu(x)
        x = self.pool3(x)

        x = self.conv4_1(x)
        x = self.bn4_1(x)
        x = self.relu(x)
        x = self.conv4_2(x)
        x = self.bn4_2(x)
        x = self.relu(x)
        x = self.pool4(x)

        x = self.conv5_1(x)
        x = self.bn5_1(x)
        x = self.relu(x)
        x = self.conv5_2(x)
        x = self.bn5_2(x)
        x = self.relu(x)

        # Fully Connected Classifier Part
        x = self.gap(x)
        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        return x

In [8]:
# Network
model = VGG(num_classes=100)

# Random input
x = torch.randn((1, 3, 32, 32))

# Forward
out = model(x)

# Check the output shape
print("Output tensor shape is :", out.shape)

Output tensor shape is : torch.Size([1, 100])


In [9]:
model = VGG().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=1e-4)

In [10]:
def eval_func(model,data_iter,device):
    model.eval()
    with torch.no_grad():
        count, total_count = 0,0
        for batch_in, batch_label in data_iter:
            x = batch_in.to(device)
            y = batch_label.to(device)
            y_pred = model.forward(x)
            y_ = torch.argmax(y_pred, dim=-1)
            count += (y==y_).sum().item()
            total_count += batch_in.size(0)
    model.train()
    return count/total_count

In [11]:
eval_func(model,train_iter,device)

0.01

In [12]:
os.makedirs('./log', exist_ok=True)
EPOCH = 20
model.train()
for epoch in range(EPOCH):
    for batch_in, batch_label in train_iter:
        x = batch_in.to(device)
        y = batch_label.to(device)
        y_pred = model.forward(x)
        optimizer.zero_grad()
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
    train_acc = eval_func(model,train_iter,device)
    test_acc = eval_func(model,test_iter,device)
    print(f'EPOCH : {epoch}, train_acc : {train_acc}, test_acc : {test_acc}')

EPOCH : 0, train_acc : 0.1633, test_acc : 0.1598
EPOCH : 1, train_acc : 0.22548, test_acc : 0.2185
EPOCH : 2, train_acc : 0.33694, test_acc : 0.327
EPOCH : 3, train_acc : 0.39842, test_acc : 0.3747
EPOCH : 4, train_acc : 0.4193, test_acc : 0.3944
EPOCH : 5, train_acc : 0.48346, test_acc : 0.4526
EPOCH : 6, train_acc : 0.48376, test_acc : 0.4373
EPOCH : 7, train_acc : 0.54828, test_acc : 0.4912
EPOCH : 8, train_acc : 0.5629, test_acc : 0.4901
EPOCH : 9, train_acc : 0.6083, test_acc : 0.5297
EPOCH : 10, train_acc : 0.63976, test_acc : 0.54
EPOCH : 11, train_acc : 0.60254, test_acc : 0.5113
EPOCH : 12, train_acc : 0.65588, test_acc : 0.5406
EPOCH : 13, train_acc : 0.67602, test_acc : 0.5492
EPOCH : 14, train_acc : 0.66472, test_acc : 0.5311
EPOCH : 15, train_acc : 0.70662, test_acc : 0.5453
EPOCH : 16, train_acc : 0.69784, test_acc : 0.5306
EPOCH : 17, train_acc : 0.7285, test_acc : 0.543
EPOCH : 18, train_acc : 0.72628, test_acc : 0.5381
EPOCH : 19, train_acc : 0.73044, test_acc : 0.531


In [ ]:
num_choice = 25
choice = np.random.choice(60000,num_choice,replace=False)
x = train_data.data[choice]
t = train_data.target[choice]